# GIS 495 Final Project Connor Bonham

---

**Overview:**

In this project I will evaluate the correlation between:

* Streamflow vs Water Temperature
* Streamflow vs Elevation
* Streamflow vs Human Wealth

(I originally wanted to use suspended sedement and land use but there are 0 stations in north carolina which measure suspended sedement, and the tif file for land use was too big to push to github)


I will also present spatial data in an interactive StoryMaps using open source products. I will also host a live version of an HTML webpage on GitHub Pages.


---


This code was developed from `storymap.ipynb` of [this Github repository](https://github.com/mdgaines/leaflet_storymap_2023)

That code is adapted from the `index.html` file from [this GitHub repository](https://github.com/HandsOnDataViz/leaflet-map-simple).

That exercise is an adaptation of the ["Leaflet Storymaps with Google Sheets" tutorial](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html) from the *Hands-On Data Visualization* book by Jack Dougherty & Ilya Ilyankou. The open-acess web edition of the book can be found [here](https://handsondataviz.org/).

Dougherty and Ilyankou developed Leaflet Storymaps with Google Sheets to create an open-source, easily migratable, story map tool. 
I used Mollie D. Gaines's adaptation as a basis for this project.

In [5]:
import pandas as pd
import geopandas as gpd
import folium
import csv
import hydrofunctions as hf
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import rasterio as rio
import rioxarray as rxr
import os

# Narrative on the Rationale and Objective of the Work
tbd, needs 5 sources

Creating Options CSV

In [6]:
# read in the options csv as a data frame
options_df = pd.read_csv('./data/Options_Template.csv', index_col=0)

In [8]:
# Personalize Options via indexing
options_df['Customize']['Author Name'] = 'Connor Bonham'
options_df['Customize']['Author Github Repo Link'] = 'https://github.com/ConBonAon/FinalProject2023'
options_df['Customize']['Storymap Title'] = 'Bonham Final Project 2023'
options_df['Customize']['Storymap Subtitle'] = 'GIS 495, Connor Bonham'
options_df['Customize']['Author Email or Website'] = 'connorallenbon@gmail.com'
options_df['Customize']['Narrative Background Color'] = 'lightgray'
# save options as Options.csv
options_df.to_csv('./data/Options.csv')

In [9]:
options_df.head()

,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
Storymap Title,Bonham Final Project 2023,NaN
Storymap Subtitle,"GIS 495, Connor Bonham","Add your subtitle, or delete that portion"
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29


# Reading in Data

## Locations
In North Carolina 37 USGS stations measure for water temperature.

![Temp](./imgs/Water_temp_stations.JPG)

2 are out of commission and 9 are seasonal.

![nonseasonal](./imgs/247Stations.JPG)

Which leaves 26 stations for data gathering.


[1](https://waterdata.usgs.gov/monitoring-location/0351706800/#parameterCode=00065&period=P7D&showMedian=true), [2](https://waterdata.usgs.gov/monitoring-location/03512000/#parameterCode=00065&period=P7D&showMedian=true), [3](https://waterdata.usgs.gov/monitoring-location/03460000/#parameterCode=00065&period=P7D&showMedian=true), [4](https://waterdata.usgs.gov/monitoring-location/03455500/#parameterCode=00065&period=P7D&showMedian=true), [5](https://waterdata.usgs.gov/monitoring-location/03456991/#parameterCode=00065&period=P7D&showMedian=true), [6](https://waterdata.usgs.gov/monitoring-location/03441000/#parameterCode=00065&period=P7D&showMedian=true), [7](https://waterdata.usgs.gov/monitoring-location/03446000/#parameterCode=00065&period=P7D&showMedian=true), [8](https://waterdata.usgs.gov/monitoring-location/03447687/#parameterCode=00065&period=P7D&showMedian=true), [9](https://waterdata.usgs.gov/monitoring-location/03447890/#parameterCode=00010&period=P7D&showMedian=true), [10](https://waterdata.usgs.gov/monitoring-location/03448050/#parameterCode=00010&period=P7D&showMedian=true), [11](https://waterdata.usgs.gov/monitoring-location/0344878100/#parameterCode=00065&period=P7D&showMedian=true), [12](https://waterdata.usgs.gov/monitoring-location/03451200/#parameterCode=00010&period=P7D&showMedian=true), [13](https://waterdata.usgs.gov/monitoring-location/03451500/#parameterCode=00065&period=P7D&showMedian=true), [14](https://waterdata.usgs.gov/monitoring-location/0344894205/#parameterCode=00065&period=P7D&showMedian=true), [15](https://waterdata.usgs.gov/monitoring-location/02137727/#parameterCode=00065&period=P7D&showMedian=true), [16](https://waterdata.usgs.gov/monitoring-location/02111000/#parameterCode=00065&period=P7D&showMedian=true), [17](https://waterdata.usgs.gov/monitoring-location/02101726/#parameterCode=00065&period=P7D&showMedian=true), [18](https://waterdata.usgs.gov/monitoring-location/02077200/#parameterCode=00065&period=P7D&showMedian=true), [19](https://waterdata.usgs.gov/monitoring-location/02077303/#parameterCode=00065&period=P7D&showMedian=true), [20](https://waterdata.usgs.gov/monitoring-location/02087182/#parameterCode=00010&period=P7D&showMedian=true), [21](https://waterdata.usgs.gov/monitoring-location/02102500/#parameterCode=00065&period=P7D&showMedian=true), [22](https://waterdata.usgs.gov/monitoring-location/02105769/#parameterCode=00065&period=P7D&showMedian=true), [23](https://waterdata.usgs.gov/monitoring-location/0208062765/#parameterCode=00065&period=P7D&showMedian=true), [24](https://waterdata.usgs.gov/monitoring-location/02081094/#parameterCode=00065&period=P7D&showMedian=true), [25](https://waterdata.usgs.gov/monitoring-location/0208114150/#parameterCode=00065&period=P7D&showMedian=true), [26](https://waterdata.usgs.gov/monitoring-location/0208458893/#parameterCode=00010&period=P7D&showMedian=true)

### Requesting Data from USGS

I requested 2022 daily data for all 26 locations assuming that they all measured both water temp and streamflow which turned out to be false.

In [10]:
# general objects
site_no = '0351706800,03512000,03460000,03455500,03456991,03441000,03446000,03447687,03447890,03448050,0344878100,03451200,03451500,0344894205,02137727,02111000,02101726,02077200,02077303,02087182,02102500,02105769,0208062765,02081094,0208114150,0208458893' #site no
service = 'dv'
st_date, end_date = '2022-01-01', '2023-01-01'

# flow in ft3/s
daily_mean = hf.NWIS(site=site_no, service=service, start_date=st_date, end_date=end_date)
# What are the properties of the dataset? You can see below that there are two types of data being collected at our site
daily_mean

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&sites=0351706800%2C03512000%2C03460000%2C03455500%2C03456991%2C03441000%2C03446000%2C03447687%2C03447890%2C03448050%2C0344878100%2C03451200%2C03451500%2C0344894205%2C02137727%2C02111000%2C02101726%2C02077200%2C02077303%2C02087182%2C02102500%2C02105769%2C0208062765%2C02081094%2C0208114150%2C0208458893&startDT=2022-01-01&endDT=2023-01-01


USGS:02077200: HYCO CREEK NEAR LEASBURG, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00060: <Day>  Discharge, cubic feet per second 
    00065: <Day>  Gage height, feet 
USGS:02077303: HYCO R BL ABAY D NR MCGEHEES MILL, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00060: <Day>  Discharge, cubic feet per second 
    00065: <Day>  Gage height, feet 
USGS:0208062765: ROANOKE RIVER AT HALIFAX, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00065: <Day>  Gage height, feet 
    00095: <Day>  Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius 
    00300: <Day>  Dissolved oxygen, water, unfiltered, milligrams per liter 
    00400: <Day>  pH, water, unfiltered, field, standard units 
USGS:02081094: ROANOKE RIVER AT JAMESVILLE, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00065: <Day>  Gage height, feet 
    00095: <Day>  Specific conductance, water, unfiltered, microsiemens per centimeter 

In [11]:
daily_mean.df()
daily_mean.df().head()

,USGS:02077200:00010:00001,USGS:02077200:00010:00001_qualifiers,USGS:02077200:00010:00002,USGS:02077200:00010:00002_qualifiers,USGS:02077200:00010:00003,USGS:02077200:00010:00003_qualifiers,USGS:02077200:00060:00003,USGS:02077200:00060:00003_qualifiers,USGS:02077200:00065:00003,USGS:02077200:00065:00003_qualifiers,...,USGS:0351706800:00010:00002,USGS:0351706800:00010:00002_qualifiers,USGS:0351706800:00010:00003,USGS:0351706800:00010:00003_qualifiers,USGS:0351706800:00045:00006,USGS:0351706800:00045:00006_qualifiers,USGS:0351706800:00060:00003,USGS:0351706800:00060:00003_qualifiers,USGS:0351706800:00065:00003,USGS:0351706800:00065:00003_qualifiers
datetimeUTC,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,16.6,A,14.0,A,15.1,A,3.06,A,23.66,A,...,12.4,A,13.3,A,0.21,A,132.0,A,2.97,A
2022-01-02 00:00:00+00:00,16.5,A,15.8,A,16.2,A,5.34,A,23.78,A,...,12.6,A,13.7,A,NaN,hf.missing,329.0,A,3.63,A
2022-01-03 00:00:00+00:00,15.8,A,8.4,A,11.5,A,194.00,A,26.38,A,...,7.4,A,8.9,A,NaN,hf.missing,289.0,A,3.55,A
2022-01-04 00:00:00+00:00,8.4,A,6.0,A,6.8,A,170.00,A,26.31,A,...,6.3,A,6.9,A,NaN,hf.missing,209.0,A,3.30,A
2022-01-05 00:00:00+00:00,7.3,A,5.5,A,6.3,A,52.70,A,24.94,A,...,6.8,A,7.5,A,NaN,hf.missing,185.0,A,3.21,A


In [12]:
streamflow_values = daily_mean.df('q')
streamflow_values

,USGS:02077200:00060:00003,USGS:02077303:00060:00003,USGS:02101726:00060:00003,USGS:02102500:00060:00003,USGS:02105769:00060:00003,USGS:02111000:00060:00003,USGS:02137727:00060:00003,USGS:03441000:00060:00003,USGS:03446000:00060:00003,USGS:03447687:00060:00003,USGS:0344878100:00060:00003,USGS:0344894205:00060:00003,USGS:03451500:00060:00003,USGS:03455500:00060:00003,USGS:03456991:00060:00003,USGS:03460000:00060:00003,USGS:03512000:00060:00003,USGS:0351706800:00060:00003
datetimeUTC,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,3.06,3.57,3.61,853.0,1580.0,16.3,116.0,64.3,83.0,846.0,85.8,12.5,1070.0,44.9,157.0,70.4,398.0,132.0
2022-01-02 00:00:00+00:00,5.34,3.62,9.63,899.0,1990.0,34.6,175.0,135.0,144.0,1330.0,190.0,49.4,1620.0,162.0,410.0,477.0,1760.0,329.0
2022-01-03 00:00:00+00:00,194.00,4.83,1140.00,6830.0,3090.0,93.0,445.0,127.0,168.0,2180.0,329.0,54.6,2970.0,116.0,502.0,340.0,1300.0,289.0
2022-01-04 00:00:00+00:00,170.00,4.86,261.00,18100.0,6440.0,46.2,240.0,98.7,131.0,1740.0,172.0,37.4,2370.0,82.5,351.0,228.0,924.0,209.0
2022-01-05 00:00:00+00:00,52.70,4.31,71.80,18800.0,13100.0,32.2,185.0,87.0,115.0,1350.0,138.0,30.4,1760.0,72.6,298.0,179.0,754.0,185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,30.60,13.50,18.60,3940.0,5780.0,40.9,181.0,106.0,138.0,1250.0,85.3,20.6,1370.0,73.2,221.0,69.6,305.0,101.0
2022-12-29 00:00:00+00:00,25.90,13.40,16.00,3440.0,5280.0,39.8,177.0,105.0,134.0,1210.0,82.2,20.4,1330.0,75.7,219.0,66.7,296.0,102.0
2022-12-30 00:00:00+00:00,23.40,13.30,13.70,1690.0,4920.0,40.1,178.0,106.0,133.0,1200.0,82.5,21.0,1300.0,78.4,218.0,64.3,292.0,101.0


In [13]:
Watertemp_Values = daily_mean.df('00010')
Watertemp_Values.head()

,USGS:02077200:00010:00001,USGS:02077200:00010:00002,USGS:02077200:00010:00003,USGS:02077303:00010:00001,USGS:02077303:00010:00002,USGS:02077303:00010:00003,USGS:0208062765:00010:00001,USGS:0208062765:00010:00002,USGS:0208062765:00010:00003,USGS:02081094:00010:00001,...,USGS:03456991:00010:00003,USGS:03460000:00010:00001,USGS:03460000:00010:00002,USGS:03460000:00010:00003,USGS:03512000:00010:00001,USGS:03512000:00010:00002,USGS:03512000:00010:00003,USGS:0351706800:00010:00001,USGS:0351706800:00010:00002,USGS:0351706800:00010:00003
datetimeUTC,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,16.6,14.0,15.1,12.4,11.0,11.7,12.9,11.5,12.0,13.4,...,11.6,12.8,10.2,11.6,13.4,11.0,11.9,14.5,12.4,13.3
2022-01-02 00:00:00+00:00,16.5,15.8,16.2,12.9,12.4,12.6,14.1,12.9,13.8,13.7,...,12.6,12.8,10.6,11.6,13.8,11.5,12.5,14.5,12.6,13.7
2022-01-03 00:00:00+00:00,15.8,8.4,11.5,13.3,11.0,11.7,14.2,10.4,12.6,13.9,...,8.9,10.6,5.9,7.8,11.5,6.5,8.9,12.6,7.4,8.9
2022-01-04 00:00:00+00:00,8.4,6.0,6.8,11.0,9.9,10.4,11.6,10.5,11.3,12.9,...,5.9,5.9,4.5,5.3,6.5,5.6,6.0,7.4,6.3,6.9
2022-01-05 00:00:00+00:00,7.3,5.5,6.3,10.0,9.7,9.9,11.1,10.2,10.5,12.4,...,6.1,6.6,5.1,5.7,7.2,5.2,6.1,8.4,6.8,7.5


In [14]:
#making watertemp values the mean version no the min or max
filtered_temp_columns = [col for col in Watertemp_Values.columns if '00003' in col]
Avg_Watertemp_Values = Watertemp_Values[filtered_temp_columns]
Avg_Watertemp_Values

,USGS:02077200:00010:00003,USGS:02077303:00010:00003,USGS:0208062765:00010:00003,USGS:02081094:00010:00003,USGS:0208458893:00010:00003,USGS:02087182:00010:00003,USGS:02101726:00010:00003,USGS:02111000:00010:00003,USGS:02137727:00010:00003,USGS:03446000:00010:00003,USGS:03447687:00010:00003,USGS:03447890:00010:00003,USGS:03448050:00010:00003,USGS:0344878100:00010:00003,USGS:03451200:00010:00003,USGS:03451500:00010:00003,USGS:03456991:00010:00003,USGS:03460000:00010:00003,USGS:03512000:00010:00003,USGS:0351706800:00010:00003
datetimeUTC,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,15.1,11.7,12.0,12.8,16.8,13.3,12.5,12.6,12.0,11.9,13.2,13.2,13.2,11.9,12.8,12.8,11.6,11.6,11.9,13.3
2022-01-02 00:00:00+00:00,16.2,12.6,13.8,13.6,17.5,13.1,13.5,14.1,13.6,13.1,14.1,14.0,14.0,13.2,13.9,13.9,12.6,11.6,12.5,13.7
2022-01-03 00:00:00+00:00,11.5,11.7,12.6,13.6,15.8,12.2,12.5,9.8,11.1,9.5,11.5,11.3,11.5,9.6,11.3,11.0,8.9,7.8,8.9,8.9
2022-01-04 00:00:00+00:00,6.8,10.4,11.3,12.5,10.0,11.5,10.7,5.7,7.2,5.3,8.3,8.5,8.6,5.9,8.3,8.1,5.9,5.3,6.0,6.9
2022-01-05 00:00:00+00:00,6.3,9.9,10.5,11.9,10.1,11.4,9.8,5.7,6.7,5.3,7.3,7.4,7.5,5.7,7.3,7.2,6.1,5.7,6.1,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,1.4,5.2,6.5,5.5,2.8,7.4,3.2,1.7,2.4,1.2,2.1,1.8,1.9,0.4,1.6,1.7,1.3,-0.0,1.0,3.8
2022-12-29 00:00:00+00:00,2.1,5.5,6.6,5.4,4.2,7.3,3.5,2.6,3.0,2.2,3.0,2.8,2.8,1.9,2.5,2.5,2.4,0.7,2.9,5.1
2022-12-30 00:00:00+00:00,3.6,5.8,6.8,5.6,5.5,7.2,4.0,3.8,4.3,3.4,4.2,3.9,3.9,3.5,3.7,3.7,3.2,2.8,4.1,6.6


20 locations have avg temp, 18 have avg streamflow. 

In [15]:
potential_locations = ["0351706800", "03512000", "03460000", "03455500", "03456991", "03441000", "03446000",
                       "03447687", "03447890", "03448050", "0344878100", "03451200", "03451500", "0344894205",
                       "02137727", "02111000", "02101726", "02077200", "02077303", "02087182", "02102500",
                       "02105769", "0208062765", "02081094", "0208114150", "0208458893"]
# Extract location numbers from both datasets
streamflow_locations = [col.split(":")[1] for col in streamflow_values.columns if 'USGS:' in col and '00060' in col]
watertemp_locations = [col.split(":")[1] for col in Avg_Watertemp_Values.columns if 'USGS:' in col and '00010' in col]

# Find common locations in both datasets
common_locations = set(streamflow_locations).intersection(watertemp_locations)

# Construct column names for the common locations in both datasets
filtered_streamflow_columns = ['USGS:' + loc + ':00060:00003' for loc in common_locations]
filtered_watertemp_columns = ['USGS:' + loc + ':00010:00003' for loc in common_locations]

# Filter columns in both datasets based on the common locations
filtered_streamflow_df = streamflow_values[filtered_streamflow_columns]
filtered_watertemp_df = Avg_Watertemp_Values[filtered_watertemp_columns]



In [16]:
# Display the resulting DataFrames
filtered_streamflow_df

,USGS:02101726:00060:00003,USGS:03460000:00060:00003,USGS:0344878100:00060:00003,USGS:03447687:00060:00003,USGS:0351706800:00060:00003,USGS:02077303:00060:00003,USGS:03456991:00060:00003,USGS:02137727:00060:00003,USGS:03512000:00060:00003,USGS:03446000:00060:00003,USGS:03451500:00060:00003,USGS:02077200:00060:00003,USGS:02111000:00060:00003
datetimeUTC,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,3.61,70.4,85.8,846.0,132.0,3.57,157.0,116.0,398.0,83.0,1070.0,3.06,16.3
2022-01-02 00:00:00+00:00,9.63,477.0,190.0,1330.0,329.0,3.62,410.0,175.0,1760.0,144.0,1620.0,5.34,34.6
2022-01-03 00:00:00+00:00,1140.00,340.0,329.0,2180.0,289.0,4.83,502.0,445.0,1300.0,168.0,2970.0,194.00,93.0
2022-01-04 00:00:00+00:00,261.00,228.0,172.0,1740.0,209.0,4.86,351.0,240.0,924.0,131.0,2370.0,170.00,46.2
2022-01-05 00:00:00+00:00,71.80,179.0,138.0,1350.0,185.0,4.31,298.0,185.0,754.0,115.0,1760.0,52.70,32.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,18.60,69.6,85.3,1250.0,101.0,13.50,221.0,181.0,305.0,138.0,1370.0,30.60,40.9
2022-12-29 00:00:00+00:00,16.00,66.7,82.2,1210.0,102.0,13.40,219.0,177.0,296.0,134.0,1330.0,25.90,39.8
2022-12-30 00:00:00+00:00,13.70,64.3,82.5,1200.0,101.0,13.30,218.0,178.0,292.0,133.0,1300.0,23.40,40.1


In [17]:
filtered_watertemp_df

,USGS:02101726:00010:00003,USGS:03460000:00010:00003,USGS:0344878100:00010:00003,USGS:03447687:00010:00003,USGS:0351706800:00010:00003,USGS:02077303:00010:00003,USGS:03456991:00010:00003,USGS:02137727:00010:00003,USGS:03512000:00010:00003,USGS:03446000:00010:00003,USGS:03451500:00010:00003,USGS:02077200:00010:00003,USGS:02111000:00010:00003
datetimeUTC,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,12.5,11.6,11.9,13.2,13.3,11.7,11.6,12.0,11.9,11.9,12.8,15.1,12.6
2022-01-02 00:00:00+00:00,13.5,11.6,13.2,14.1,13.7,12.6,12.6,13.6,12.5,13.1,13.9,16.2,14.1
2022-01-03 00:00:00+00:00,12.5,7.8,9.6,11.5,8.9,11.7,8.9,11.1,8.9,9.5,11.0,11.5,9.8
2022-01-04 00:00:00+00:00,10.7,5.3,5.9,8.3,6.9,10.4,5.9,7.2,6.0,5.3,8.1,6.8,5.7
2022-01-05 00:00:00+00:00,9.8,5.7,5.7,7.3,7.5,9.9,6.1,6.7,6.1,5.3,7.2,6.3,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,3.2,-0.0,0.4,2.1,3.8,5.2,1.3,2.4,1.0,1.2,1.7,1.4,1.7
2022-12-29 00:00:00+00:00,3.5,0.7,1.9,3.0,5.1,5.5,2.4,3.0,2.9,2.2,2.5,2.1,2.6
2022-12-30 00:00:00+00:00,4.0,2.8,3.5,4.2,6.6,5.8,3.2,4.3,4.1,3.4,3.7,3.6,3.8


I now have 13 locations where both variables are measured. I want to rename the columns to the same thing so I can compare them by column.

In [18]:
#cutting out USGS: from column names
filtered_streamflow_df.columns = filtered_streamflow_df.columns.str[5:]
filtered_watertemp_df.columns = filtered_watertemp_df.columns.str[5:]




In [19]:
#cutting out :000X0:00003 from column names
filtered_streamflow_df.columns = filtered_streamflow_df.columns.str[:-12]
filtered_watertemp_df.columns = filtered_watertemp_df.columns.str[:-12]

Now that I have the columns names match, I will get a correlation coefficient for streamflow vs water temp for each location. 

In [20]:
# Extract the common column names
common_columns = set(filtered_streamflow_df.columns) & set(filtered_watertemp_df.columns)

# Initialize an empty list to store DataFrames
corrcoef_stmflw_wtrtmp_df = []

# Compute the correlation coefficient for each common column
for column in common_columns:
    data1_column = filtered_streamflow_df[column].values
    data2_column = filtered_watertemp_df[column].values

    correlation_coefficient = np.corrcoef(data1_column, data2_column)[0, 1]

    # Create a temporary DataFrame for each iteration
    temp_df = pd.DataFrame({"Location": [column], "Correlation Coefficient": [correlation_coefficient]})
    
    # Append the temporary DataFrame to the list
    corrcoef_stmflw_wtrtmp_df.append(temp_df)

    print(f"Correlation coefficient for USGS location '{column}' = {correlation_coefficient}")

# Concatenate the list of DataFrames into a single DataFrame
corrcoef_stmflw_wtrtmp_df = pd.concat(corrcoef_stmflw_wtrtmp_df, ignore_index=True)


Correlation coefficient for USGS location '02101726' = -0.15057422849004087
Correlation coefficient for USGS location '03460000' = -0.18623982827422705
Correlation coefficient for USGS location '0344878100' = nan
Correlation coefficient for USGS location '03447687' = -0.05626917624481335
Correlation coefficient for USGS location '0351706800' = -0.11225280087809647
Correlation coefficient for USGS location '02077303' = -0.13350953287047415
Correlation coefficient for USGS location '03456991' = nan
Correlation coefficient for USGS location '02137727' = -0.1205493623903371
Correlation coefficient for USGS location '03512000' = nan
Correlation coefficient for USGS location '03446000' = -0.006663145379915638
Correlation coefficient for USGS location '03451500' = -0.07493103512086549
Correlation coefficient for USGS location '02077200' = nan
Correlation coefficient for USGS location '02111000' = -0.09153856543663072


In [21]:
# Print the resulting DataFrame
print(corrcoef_stmflw_wtrtmp_df)

      Location  Correlation Coefficient
0     02101726                -0.150574
1     03460000                -0.186240
2   0344878100                      NaN
3     03447687                -0.056269
4   0351706800                -0.112253
5     02077303                -0.133510
6     03456991                      NaN
7     02137727                -0.120549
8     03512000                      NaN
9     03446000                -0.006663
10    03451500                -0.074931
11    02077200                      NaN
12    02111000                -0.091539


In [22]:
print(common_locations)

{'02101726', '03460000', '0344878100', '03447687', '0351706800', '02077303', '03456991', '02137727', '03512000', '03446000', '03451500', '02077200', '02111000'}


In [23]:
com_loc = "'03447687', '03446000', '02077200', '03451500', '03456991', '03512000', '0344878100', '02137727', '02111000', '02077303', '0351706800', '03460000', '02101726'"

# Replace single quotes with an empty string
com_loc_mod = com_loc.replace("'", "")

print(com_loc_mod)

03447687, 03446000, 02077200, 03451500, 03456991, 03512000, 0344878100, 02137727, 02111000, 02077303, 0351706800, 03460000, 02101726


Alright so the locations I am going to compare are '03447687', '03446000', '02077200', '03451500', '03456991', '03512000', '0344878100', '02137727', '02111000', '02077303', '0351706800', '03460000', '02101726' and I want to get geospacial data for these locations. 

In [24]:
Com_Loc_df = []

for column in common_columns:
    output = hf.site_file(column)
    site_table = output.table
    latitude = site_table['dec_lat_va'].iloc[0]  
    longitude = site_table['dec_long_va'].iloc[0]
    datum = site_table['dec_coord_datum_cd'].iloc[0]
    alt_land_surf = site_table['alt_va'].iloc[0]
    county = site_table['county_cd'].iloc[0]
    temp_df = pd.DataFrame({"Location": [column], "Lat": [latitude],"Long":[longitude], "Datum":[datum], "County":[county], "Elevation":[alt_land_surf]})
    Com_Loc_df.append(temp_df)
Com_Loc_df = pd.concat(Com_Loc_df, ignore_index=True)
Com_Loc_df

Retrieved the site file for site #02101726 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=02101726&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #03460000 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=03460000&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #0344878100 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=0344878100&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #03447687 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=03447687&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #0351706800 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=0351706800&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #02077303 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=02077303&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #03456991 from https://waterservices.usgs.gov/n

,Location,Lat,Long,Datum,County,Elevation
0,02101726,35.735141,-79.423075,NAD83,37,482.90
1,03460000,35.667222,-83.072778,NAD83,87,2457.30
2,0344878100,35.561389,-82.596944,NAD83,21,1975.32
3,03447687,35.429167,-82.552500,NAD83,89,2032.76
4,0351706800,35.438333,-83.918889,NAD83,75,1271.70
5,02077303,36.522722,-78.997056,NAD83,145,341.97
6,03456991,35.521944,-82.848056,NAD83,87,2581.60
7,02137727,35.685833,-82.060278,NAD83,111,1233.81
8,03512000,35.461389,-83.353611,NAD83,173,1843.25
9,03446000,35.398056,-82.595000,NAD83,89,2088.39


In [25]:
County_codes = pd.read_csv('./data/County_codes.csv')
County_codes


,County,Name
0,1,Alamance County
1,3,Alexander County
2,5,Alleghany County
3,7,Anson County
4,9,Ashe County
...,...,...
95,191,Wayne County
96,193,Wilkes County
97,195,Wilson County
98,197,Yadkin County


In [26]:

# Merge the datasets on 'county_code'
Com_Loc_County = pd.merge(Com_Loc_df, County_codes, on='County', how='left')

# Display the result
Com_Loc_County

,Location,Lat,Long,Datum,County,Elevation,Name
0,02101726,35.735141,-79.423075,NAD83,37,482.90,Chatham County
1,03460000,35.667222,-83.072778,NAD83,87,2457.30,Haywood County
2,0344878100,35.561389,-82.596944,NAD83,21,1975.32,Buncombe County
3,03447687,35.429167,-82.552500,NAD83,89,2032.76,Henderson County
4,0351706800,35.438333,-83.918889,NAD83,75,1271.70,Graham County
5,02077303,36.522722,-78.997056,NAD83,145,341.97,Person County
6,03456991,35.521944,-82.848056,NAD83,87,2581.60,Haywood County
7,02137727,35.685833,-82.060278,NAD83,111,1233.81,Mcdowell County
8,03512000,35.461389,-83.353611,NAD83,173,1843.25,Swain County
9,03446000,35.398056,-82.595000,NAD83,89,2088.39,Henderson County


## Land Cover

Since the Land cover map is from 2019 I want to grab streamflow data from 2019 to compare it to. I will use the same 13 stations from before for simplicity.

In [27]:
Strmflw_an = []
temp_df = pd.DataFrame(columns=["Location", "Year", "Streamflow"])  # Initialize outside the loop

for column in common_columns:
    output = hf.stats(column, 'annual', missingData='on')
    site_table = output.table
    
    # Filter rows where parameter_cd is '00060' and year_nu is 2019
    filtered_table = site_table.loc[(site_table['parameter_cd'] == '00060') & (site_table['year_nu'] == 2019)]
    
    # Check if there are rows in the filtered_table
    if not filtered_table.empty:
        year = filtered_table['year_nu'].iloc[0]
        stmflw = filtered_table['mean_va'].iloc[0]
        
        # Append the data for the current iteration to temp_df
        temp_df = pd.concat([temp_df, pd.DataFrame({"Location": [column], "Year": [year], "Streamflow": [stmflw]})], ignore_index=True)
    else:
        print(f"No data for {column}")

# final DataFrame
Strmflw_an = temp_df
print(Strmflw_an)


Retrieved annual statistics for site #02101726 from https://waterservices.usgs.gov/nwis/stat//


C:\Users\conno\AppData\Local\Temp\ipykernel_19100\1450280576.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.concat([temp_df, pd.DataFrame({"Location": [column], "Year": [year], "Streamflow": [stmflw]})], ignore_index=True)


Retrieved annual statistics for site #03460000 from https://waterservices.usgs.gov/nwis/stat/
Retrieved annual statistics for site #0344878100 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03447687 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #0351706800 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #02077303 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03456991 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #02137727 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03512000 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03446000 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03451500 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #02077200 

With this annual streamflow I can look at how each locations streamflow compared to their elevation via a corrcoef.

In [28]:
# Assuming Strmflw_an = df1 is the first dataset and Com_Loc_County = df2 is the second dataset
# with 'Location' as the common column
# 'Streamflow' is the column in df1, and 'Elevation' is the column in df2

# Merge the two datasets on the 'locations' column
Elv_stmfl_df = pd.merge(Strmflw_an, Com_Loc_County, on='Location', how='inner')

# Calculate the correlation coefficient
corrcoeff_Elv_stmfl = np.corrcoef(Elv_stmfl_df['Streamflow'], Elv_stmfl_df['Elevation'])[0, 1]

corrcoeff_Elv_stmfl


0.33543646774654784

When I went to push the code for reading in the land use map, it would not let me push it to git because the tif file was too large. So I abandoned that aspect of the project.

### Chapters

We will add content to our map by creating a `Chapters.csv`.

However, first we will look through our data to get the information we are interested in.

We can use the `folium` package to see what our Leaflet map will look like.

In [29]:
# Create a map centered at the mean latitude and longitude
map_center = [Com_Loc_df['Lat'].mean(), Com_Loc_df['Long'].mean()]
loc_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each location
for index, row in Com_Loc_df.iterrows():
    folium.Marker([row['Lat'], row['Long']], popup=row['Location']).add_to(loc_map)

# Save the map as an HTML file or display it in a Jupyter notebook
map_file_name='loc_map.html'

if not os.path.exists(map_file_name):
    # Save the map as an HTML file
    loc_map.save(map_file_name)
    print(f"Map saved as {map_file_name}")
else:
    print(f"A map file named {map_file_name} already exists. Not saving a new map.")
loc_map

Map saved as loc_map.html


In [30]:

header = ['Chapter', 'Media Link', 'Media Credit', 'Media Credit Link', 'Description', \
    'Zoom', 'Marker', 'Marker Color', 'Location', 'Latitude', 'Longitude', 'Overlay', 'Overlay Transparency', \
    'GeoJSON Overlay', 'GeoJSON Feature Properties']



In [32]:
# Create a new Chapters.csv

# opens a csv file in write mode
# the with statement makes sure the file closes properly when we're done
with open('./data/Chapters.csv', 'w', newline='') as options:
    # create the csv writer
    writer = csv.writer(options)

    # write a row to the csv file
    writer.writerow(header)

    writer.writerow(['Title'])

    for column in common_columns:
        writer.writerow([column])

In [34]:
chapter_df = pd.read_csv('./data/Chapters.csv')
chapter_df

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02101726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0344878100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03447687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0351706800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,02077303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,03456991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,02137727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,03512000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
chapter_df.columns

Index(['Chapter', 'Media Link', 'Media Credit', 'Media Credit Link',
       'Description', 'Zoom', 'Marker', 'Marker Color', 'Location', 'Latitude',
       'Longitude', 'Overlay', 'Overlay Transparency', 'GeoJSON Overlay',
       'GeoJSON Feature Properties'],
      dtype='object')